In [93]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

# Import language list file

In [10]:
language_list_df = pd.read_csv("AllLanguageList.csv").dropna()
language_link_df = language_list_df.drop(columns=['CategoryTreeItem', 'CategoryTreeItem 3']).rename(columns={"CategoryTreeItem 2": "language", "CategoryTreeItem href": "language_link"}).copy()

In [11]:
language_link_df

,language,language_link
0,A'ou language,https://en.wiktionary.org/wiki/Category:A%27ou...
1,A-Hmao language,https://en.wiktionary.org/wiki/Category:A-Hmao...
2,A-Pucikwar language,https://en.wiktionary.org/wiki/Category:A-Puci...
3,Aari language,https://en.wiktionary.org/wiki/Category:Aari_l...
4,Aasax language,https://en.wiktionary.org/wiki/Category:Aasax_...
...,...,...
5389,ǀXam language,https://en.wiktionary.org/wiki/Category:%C7%80...
5390,ǁAni language,https://en.wiktionary.org/wiki/Category:%C7%81...
5391,ǁXegwi language,https://en.wiktionary.org/wiki/Category:%C7%81...
5392,ǃKung language,https://en.wiktionary.org/wiki/Category:%C7%83...


# Add another column: borrowed term link

In [15]:
def createBorrowedLink(row):  
    if row['language_link'] is not None and len(row['language_link'])>0:       
        return row['language_link'].replace("_language","_borrowed_terms")
language_link_df['borrowed_link'] = language_link_df.apply(lambda row: createBorrowedLink(row), axis=1)
list(language_link_df["borrowed_link"])

['https://en.wiktionary.org/wiki/Category:A%27ou_borrowed_terms',
 'https://en.wiktionary.org/wiki/Category:A-Hmao_borrowed_terms',
 'https://en.wiktionary.org/wiki/Category:A-Pucikwar_borrowed_terms',
 'https://en.wiktionary.org/wiki/Category:Aari_borrowed_terms',
 'https://en.wiktionary.org/wiki/Category:Aasax_borrowed_terms',
 'https://en.wiktionary.org/wiki/Category:Aba_borrowed_terms',
 'https://en.wiktionary.org/wiki/Category:Abaga_borrowed_terms',
 'https://en.wiktionary.org/wiki/Category:Abai_borrowed_terms',
 'https://en.wiktionary.org/wiki/Category:Abai_Sungai_borrowed_terms',
 'https://en.wiktionary.org/wiki/Category:Abanyom_borrowed_terms',
 'https://en.wiktionary.org/wiki/Category:Abau_borrowed_terms',
 'https://en.wiktionary.org/wiki/Category:Abaza_borrowed_terms',
 'https://en.wiktionary.org/wiki/Category:Abenaki_borrowed_terms',
 'https://en.wiktionary.org/wiki/Category:Abenlen_Ayta_borrowed_terms',
 'https://en.wiktionary.org/wiki/Category:Abidji_borrowed_terms',
 'htt

In [26]:
def CheckHasLoanwords(row):  
    ipa_page = requests.get(row['borrowed_link'])
    ipa_soup = BeautifulSoup(ipa_page.content, 'html.parser')
    for ipa in ipa_soup.find_all("div", {'class': 'plainlinks'}):
        if "Wiktionary does not yet have a category for" in ipa.text:
            return False
    for ipa in ipa_soup.find_all("div", {'class': 'CategoryTreeItem'}):
        if "terms borrowed from" in ipa.text:
            return True
    return False
language_link_df['has_loanwords'] = language_link_df.apply(lambda row: CheckHasLoanwords(row), axis=1)

language_link_df 
# ipa_page = requests.get("https://en.wiktionary.org/wiki/Category:Audi_borrowed_terms")
# ipa_soup = BeautifulSoup(ipa_page.content, 'html.parser')

# for ipa in ipa_soup.find_all("div", {'class': 'plainlinks'}):
#     print(ipa.text)

,language,language_link,borrowed_link,has_loanwords
0,A'ou language,https://en.wiktionary.org/wiki/Category:A%27ou...,https://en.wiktionary.org/wiki/Category:A%27ou...,False
1,A-Hmao language,https://en.wiktionary.org/wiki/Category:A-Hmao...,https://en.wiktionary.org/wiki/Category:A-Hmao...,False
2,A-Pucikwar language,https://en.wiktionary.org/wiki/Category:A-Puci...,https://en.wiktionary.org/wiki/Category:A-Puci...,False
3,Aari language,https://en.wiktionary.org/wiki/Category:Aari_l...,https://en.wiktionary.org/wiki/Category:Aari_b...,False
4,Aasax language,https://en.wiktionary.org/wiki/Category:Aasax_...,https://en.wiktionary.org/wiki/Category:Aasax_...,False
...,...,...,...,...
5389,ǀXam language,https://en.wiktionary.org/wiki/Category:%C7%80...,https://en.wiktionary.org/wiki/Category:%C7%80...,False
5390,ǁAni language,https://en.wiktionary.org/wiki/Category:%C7%81...,https://en.wiktionary.org/wiki/Category:%C7%81...,False
5391,ǁXegwi language,https://en.wiktionary.org/wiki/Category:%C7%81...,https://en.wiktionary.org/wiki/Category:%C7%81...,False
5392,ǃKung language,https://en.wiktionary.org/wiki/Category:%C7%83...,https://en.wiktionary.org/wiki/Category:%C7%83...,False


In [66]:
language_link_df.to_csv("hasLoanwords.csv")
# language_link_df = pd.read_csv("hasLoanwords.csv")

,language,language_link,borrowed_link,has_loanwords
0,A'ou language,https://en.wiktionary.org/wiki/Category:A%27ou...,https://en.wiktionary.org/wiki/Category:A%27ou...,False
1,A-Hmao language,https://en.wiktionary.org/wiki/Category:A-Hmao...,https://en.wiktionary.org/wiki/Category:A-Hmao...,False
2,A-Pucikwar language,https://en.wiktionary.org/wiki/Category:A-Puci...,https://en.wiktionary.org/wiki/Category:A-Puci...,False
3,Aari language,https://en.wiktionary.org/wiki/Category:Aari_l...,https://en.wiktionary.org/wiki/Category:Aari_b...,False
4,Aasax language,https://en.wiktionary.org/wiki/Category:Aasax_...,https://en.wiktionary.org/wiki/Category:Aasax_...,False
...,...,...,...,...
5389,ǀXam language,https://en.wiktionary.org/wiki/Category:%C7%80...,https://en.wiktionary.org/wiki/Category:%C7%80...,False
5390,ǁAni language,https://en.wiktionary.org/wiki/Category:%C7%81...,https://en.wiktionary.org/wiki/Category:%C7%81...,False
5391,ǁXegwi language,https://en.wiktionary.org/wiki/Category:%C7%81...,https://en.wiktionary.org/wiki/Category:%C7%81...,False
5392,ǃKung language,https://en.wiktionary.org/wiki/Category:%C7%83...,https://en.wiktionary.org/wiki/Category:%C7%83...,False


In [67]:
has_loanwords_df = language_link_df[language_link_df['has_loanwords'] == True]
has_loanwords_df

,language,language_link,borrowed_link,has_loanwords
11,Abaza language,https://en.wiktionary.org/wiki/Category:Abaza_...,https://en.wiktionary.org/wiki/Category:Abaza_...,True
12,Abenaki language,https://en.wiktionary.org/wiki/Category:Abenak...,https://en.wiktionary.org/wiki/Category:Abenak...,True
13,Abenlen Ayta language,https://en.wiktionary.org/wiki/Category:Abenle...,https://en.wiktionary.org/wiki/Category:Abenle...,True
18,Abkhaz language,https://en.wiktionary.org/wiki/Category:Abkhaz...,https://en.wiktionary.org/wiki/Category:Abkhaz...,True
31,Acehnese language,https://en.wiktionary.org/wiki/Category:Acehne...,https://en.wiktionary.org/wiki/Category:Acehne...,True
...,...,...,...,...
5373,Zipser German language,https://en.wiktionary.org/wiki/Category:Zipser...,https://en.wiktionary.org/wiki/Category:Zipser...,True
5376,Zoogocho Zapotec language,https://en.wiktionary.org/wiki/Category:Zoogoc...,https://en.wiktionary.org/wiki/Category:Zoogoc...,True
5378,Zou language,https://en.wiktionary.org/wiki/Category:Zou_la...,https://en.wiktionary.org/wiki/Category:Zou_bo...,True
5380,Zulu language,https://en.wiktionary.org/wiki/Category:Zulu_l...,https://en.wiktionary.org/wiki/Category:Zulu_b...,True


In [72]:
def OriginalSources(row):  
    language_page = requests.get(row['borrowed_link'])
    language_soup = BeautifulSoup(language_page.content, 'html.parser')
    source_count = []
    for lang in language_soup.find_all("div", {'class': 'CategoryTreeItem'}):
        if "terms borrowed from" in lang.text:
            source_count.append(lang.text.split("from ")[1])
    return source_count
#has_loanwords_df['original_sources'] = has_loanwords_df.apply(lambda row: OriginalSources(row), axis=1)

In [73]:
has_loanwords_df['original_sources'] = has_loanwords_df.apply(lambda row: OriginalSources(row), axis=1)
has_loanwords_df['language'] = has_loanwords_df.apply(lambda row: row['language'].lower().replace(" language",""), axis=1)
lang_expanded_df = has_loanwords_df.explode('original_sources')

/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [98]:
lang_expanded_df#.to_csv("almost_done.csv")

,language,language_link,borrowed_link,has_loanwords,original_sources
11,abaza,https://en.wiktionary.org/wiki/Category:Abaza_...,https://en.wiktionary.org/wiki/Category:Abaza_...,True,"Abkhaz‎ (0 c, 1 e)"
11,abaza,https://en.wiktionary.org/wiki/Category:Abaza_...,https://en.wiktionary.org/wiki/Category:Abaza_...,True,"Circassian languages‎ (0 c, 1 e)"
12,abenaki,https://en.wiktionary.org/wiki/Category:Abenak...,https://en.wiktionary.org/wiki/Category:Abenak...,True,"English‎ (0 c, 18 e)"
12,abenaki,https://en.wiktionary.org/wiki/Category:Abenak...,https://en.wiktionary.org/wiki/Category:Abenak...,True,"French‎ (0 c, 1 e)"
13,abenlen ayta,https://en.wiktionary.org/wiki/Category:Abenle...,https://en.wiktionary.org/wiki/Category:Abenle...,True,"Kapampangan‎ (0 c, 1 e)"
...,...,...,...,...,...
5380,zulu,https://en.wiktionary.org/wiki/Category:Zulu_l...,https://en.wiktionary.org/wiki/Category:Zulu_b...,True,"ǁXegwi‎ (0 c, 1 e)"
5393,ǃxóõ,https://en.wiktionary.org/wiki/Category:%C7%83...,https://en.wiktionary.org/wiki/Category:%C7%83...,True,"Afrikaans‎ (0 c, 1 e)"
5393,ǃxóõ,https://en.wiktionary.org/wiki/Category:%C7%83...,https://en.wiktionary.org/wiki/Category:%C7%83...,True,"English‎ (0 c, 3 e)"
5393,ǃxóõ,https://en.wiktionary.org/wiki/Category:%C7%83...,https://en.wiktionary.org/wiki/Category:%C7%83...,True,"Kgalagadi‎ (0 c, 1 e)"


In [94]:
def splitSourceColumn(item):
    source_lang = str(item).split("(")[0].strip()
    pattern = "c, (.*?) e"
    e = re.search(pattern, str(item)).group(1)
    # e = str(item).split("(")[1].split(", ")[1].split(" e")[0]
    return pd.Series([source_lang, e])

In [99]:
lang_expanded_df[['source_language','e']] = lang_expanded_df.original_sources.apply(lambda x: splitSourceColumn(x))

In [101]:
#lang_expanded_df.to_csv("loanword_source_count.csv")

In [104]:
# change column type to int
lang_expanded_df["e"]= pd.to_numeric(lang_expanded_df["e"])
lang_expanded_df.to_csv("loanword_source_count.csv")

# Get rid of extra columns

In [108]:
final_df = lang_expanded_df[["language","source_language","e"]]
final_df#.to_csv("language_e.csv")

# Extra experiment 

In [32]:
a = ["hello"]
a.append("world")
a

['hello', 'world']

In [95]:
temp_df = has_loanwords_df[0:10].copy()
#temp_df
temp_df['original_sources'] = temp_df.apply(lambda row: OriginalSources(row), axis=1)
temp_df['language'] = temp_df.apply(lambda row: row['language'].replace(" language",""), axis=1)
lastdf = temp_df.explode('original_sources').copy()
lastdf

,language,language_link,borrowed_link,has_loanwords,original_sources
11,abaza,https://en.wiktionary.org/wiki/Category:Abaza_...,https://en.wiktionary.org/wiki/Category:Abaza_...,True,"Abkhaz‎ (0 c, 1 e)"
11,abaza,https://en.wiktionary.org/wiki/Category:Abaza_...,https://en.wiktionary.org/wiki/Category:Abaza_...,True,"Circassian languages‎ (0 c, 1 e)"
12,abenaki,https://en.wiktionary.org/wiki/Category:Abenak...,https://en.wiktionary.org/wiki/Category:Abenak...,True,"English‎ (0 c, 18 e)"
12,abenaki,https://en.wiktionary.org/wiki/Category:Abenak...,https://en.wiktionary.org/wiki/Category:Abenak...,True,"French‎ (0 c, 1 e)"
13,abenlen ayta,https://en.wiktionary.org/wiki/Category:Abenle...,https://en.wiktionary.org/wiki/Category:Abenle...,True,"Kapampangan‎ (0 c, 1 e)"
...,...,...,...,...,...
69,aghwan,https://en.wiktionary.org/wiki/Category:Aghwan...,https://en.wiktionary.org/wiki/Category:Aghwan...,True,"Middle Persian‎ (0 c, 7 e)"
69,aghwan,https://en.wiktionary.org/wiki/Category:Aghwan...,https://en.wiktionary.org/wiki/Category:Aghwan...,True,"Northwestern Iranian languages‎ (0 c, 1 e)"
69,aghwan,https://en.wiktionary.org/wiki/Category:Aghwan...,https://en.wiktionary.org/wiki/Category:Aghwan...,True,"Old Armenian‎ (1 c, 25 e)"
69,aghwan,https://en.wiktionary.org/wiki/Category:Aghwan...,https://en.wiktionary.org/wiki/Category:Aghwan...,True,"Old Georgian‎ (0 c, 5 e)"


In [96]:
lastdf[['source_language','e']] = lastdf.original_sources.apply(lambda x: splitSourceColumn(x))

In [102]:
lastdf

,language,language_link,borrowed_link,has_loanwords,original_sources,source_language,e
11,abaza,https://en.wiktionary.org/wiki/Category:Abaza_...,https://en.wiktionary.org/wiki/Category:Abaza_...,True,"Abkhaz‎ (0 c, 1 e)",Abkhaz‎,1
11,abaza,https://en.wiktionary.org/wiki/Category:Abaza_...,https://en.wiktionary.org/wiki/Category:Abaza_...,True,"Circassian languages‎ (0 c, 1 e)",Circassian languages‎,1
12,abenaki,https://en.wiktionary.org/wiki/Category:Abenak...,https://en.wiktionary.org/wiki/Category:Abenak...,True,"English‎ (0 c, 18 e)",English‎,18
12,abenaki,https://en.wiktionary.org/wiki/Category:Abenak...,https://en.wiktionary.org/wiki/Category:Abenak...,True,"French‎ (0 c, 1 e)",French‎,1
13,abenlen ayta,https://en.wiktionary.org/wiki/Category:Abenle...,https://en.wiktionary.org/wiki/Category:Abenle...,True,"Kapampangan‎ (0 c, 1 e)",Kapampangan‎,1
...,...,...,...,...,...,...,...
69,aghwan,https://en.wiktionary.org/wiki/Category:Aghwan...,https://en.wiktionary.org/wiki/Category:Aghwan...,True,"Middle Persian‎ (0 c, 7 e)",Middle Persian‎,7
69,aghwan,https://en.wiktionary.org/wiki/Category:Aghwan...,https://en.wiktionary.org/wiki/Category:Aghwan...,True,"Northwestern Iranian languages‎ (0 c, 1 e)",Northwestern Iranian languages‎,1
69,aghwan,https://en.wiktionary.org/wiki/Category:Aghwan...,https://en.wiktionary.org/wiki/Category:Aghwan...,True,"Old Armenian‎ (1 c, 25 e)",Old Armenian‎,25
69,aghwan,https://en.wiktionary.org/wiki/Category:Aghwan...,https://en.wiktionary.org/wiki/Category:Aghwan...,True,"Old Georgian‎ (0 c, 5 e)",Old Georgian‎,5


In [103]:
lastdf["e"]= pd.to_numeric(lastdf["e"])
lastdf

,language,language_link,borrowed_link,has_loanwords,original_sources,source_language,e
11,abaza,https://en.wiktionary.org/wiki/Category:Abaza_...,https://en.wiktionary.org/wiki/Category:Abaza_...,True,"Abkhaz‎ (0 c, 1 e)",Abkhaz‎,1
11,abaza,https://en.wiktionary.org/wiki/Category:Abaza_...,https://en.wiktionary.org/wiki/Category:Abaza_...,True,"Circassian languages‎ (0 c, 1 e)",Circassian languages‎,1
12,abenaki,https://en.wiktionary.org/wiki/Category:Abenak...,https://en.wiktionary.org/wiki/Category:Abenak...,True,"English‎ (0 c, 18 e)",English‎,18
12,abenaki,https://en.wiktionary.org/wiki/Category:Abenak...,https://en.wiktionary.org/wiki/Category:Abenak...,True,"French‎ (0 c, 1 e)",French‎,1
13,abenlen ayta,https://en.wiktionary.org/wiki/Category:Abenle...,https://en.wiktionary.org/wiki/Category:Abenle...,True,"Kapampangan‎ (0 c, 1 e)",Kapampangan‎,1
...,...,...,...,...,...,...,...
69,aghwan,https://en.wiktionary.org/wiki/Category:Aghwan...,https://en.wiktionary.org/wiki/Category:Aghwan...,True,"Middle Persian‎ (0 c, 7 e)",Middle Persian‎,7
69,aghwan,https://en.wiktionary.org/wiki/Category:Aghwan...,https://en.wiktionary.org/wiki/Category:Aghwan...,True,"Northwestern Iranian languages‎ (0 c, 1 e)",Northwestern Iranian languages‎,1
69,aghwan,https://en.wiktionary.org/wiki/Category:Aghwan...,https://en.wiktionary.org/wiki/Category:Aghwan...,True,"Old Armenian‎ (1 c, 25 e)",Old Armenian‎,25
69,aghwan,https://en.wiktionary.org/wiki/Category:Aghwan...,https://en.wiktionary.org/wiki/Category:Aghwan...,True,"Old Georgian‎ (0 c, 5 e)",Old Georgian‎,5


In [92]:
import re
item = "Old Armenian (1 c, 25 e)"
pattern = "c, (.*?) e"
e = re.search(pattern, item).group(1)
e

'25'